In [ ]:
!pip install pandas==1.2.4

In [ ]:
!pip install unidecode

In [ ]:
import time
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
df_propiedades=pd.read_csv('/gdrive/MyDrive/Seminario/consolidado_propiedades_FR_30052021.csv', delimiter=';', encoding='latin1') # Lectura Andrés

In [ ]:
# ******************************************************************************************************************** #
## LECTURAS PARA ESTEFA:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/Seminario'
df_propiedades=pd.read_csv('consolidado_propiedades_FR_30052021.csv', delimiter=';', encoding='latin1') #Lectura Estefa
print(df_propiedades.shape)

df_muni = pd.read_csv('davipola.csv', delimiter=',')
df_muni.shape
# ******************************************************************************************************************** #

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Seminario
(2715, 188)


(1126, 5)

In [ ]:
lista_columnas=list(df_propiedades.columns)
col_limpia=[]

import unidecode

for columna in lista_columnas:
  col=columna.lower().replace('/','').replace('.','').strip().replace('  ',' ').replace('  ',' ').replace(' ', '_')
  col=unidecode.unidecode(col)
  col_limpia.append(col)

columnas_nuevas=dict(zip(lista_columnas,col_limpia))

In [ ]:
df_propiedades.rename(columns=columnas_nuevas, inplace=True)
df_propiedades.rename(columns={'area':'area_m2'}, inplace=True)

In [ ]:
df_propiedades.head(1)

,tipo,url,titulo,precio,area_m2,habitaciones,banos,garajes,descripcion,balcon,bano_de_servicio,chimenea,cocina_equipada,cuarto_de_servicio,sauna_turco_jacuzzi,servicio_de_internet,servicios_publicos,zona_de_bbq,acceso_pavimentado,garaje_parqueadero(s),porteria_vigilancia,senderos_ecologicos,area_rural,cerca_de_zona_urbana,sobre_via_secundaria,trans_publico_cercano,caractint,caractext,caractsec,estrato,antiguedad,cocina_integral,ventilacion_natural,asador,bosque_nativo,establo,invernadero,pesebrera,terraza,vista_panoramica,...,puerta_electrica,ubicada_en_edificio,finca_agricola,sensor_de_movimiento,planta_electrica,servicios_independientes,cuarto_de_escoltas,salon_de_conferencias,rociadores_de_agua,con_casa_club,disponibilidad_wifi,galpon,piso_de_alta_resistencia,cableado_de_red,puerta_de_seguridad,sala_de_internet,finca_avicola,lote_en_construccion,salon_de_videoconferencias,pasaje_comercial,sistema_de_riego,acceso_para_camiones,con_vivienda,lic_de_construccion,panoramica_un_lado,en_club,auditorio,esquinero,oficinas_administrativas,con_casa_prefabricada,en_zona_comercial,finca_cafetera,servicio_de_lavanderia,altura_restringida,ascensor(es)_inteligente(s),lote_vacio,control_de_acustica,control_termico,panoramica_360o,con_cerca_electrica
0,Usada,https://www.fincaraiz.com.co/casa-campestre-en...,Casa Campestre en Venta Rionegro Vereda Guayabito,$ 1.380.000.000,"320,00 m²",5,6,2,Propiedad moderna con muy buenos acabados. Her...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"['Balcón', 'Baño de Servicio', 'Chimenea', 'Co...","['Acceso Pavimentado', 'Garaje / Parqueadero(s...","['Área Rural', 'Cerca de Zona Urbana', 'Sobre ...",5,1 a 8 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df_muni=pd.read_csv('/gdrive/MyDrive/Seminario/davipola.csv', delimiter=',')

df_muni_antioquia=df_muni[df_muni["Nombre Departamento"]=='ANTIOQUIA']
df_muni_antioquia.head(1)

,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada
0,05,5001.0,ANTIOQUIA,MEDELLÍN,Municipio


In [ ]:
df_muni_antioquia["Nombre Municipio"]=df_muni_antioquia["Nombre Municipio"].apply(lambda x: unidecode.unidecode(x.lower()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
list_muni=np.array(df_muni_antioquia["Nombre Municipio"])
list_muni=np.append(list_muni, np.array(['ceja','viboral','llano','llanogrande','pereira','union','unión','santuario']))

In [ ]:
def extract_ubicacion(lineas):
  texto=np.array(lineas)[0]
  palabras=texto.lower().replace(',','').replace(';','').split(' ')
  muni=''
  
  for palabra in palabras:

    if palabra in list_muni:
        muni = palabra
        break
    else:
        muni = ''

  if muni == '':
    texto=np.array(lineas)[1]
    palabras=texto.lower().replace(',','').replace(';','').split(' ')
  
    for palabra in palabras:

      if palabra in list_muni:
          muni = palabra
          break
      else:

          muni = ''


  return muni

In [ ]:
tipo_propiedades=['casa','finca','apartamento','apartaestudio','lote', 'apartamentos']

def extract_tipo(lineas):

  texto=np.array(lineas)[0]

  palabras=texto.lower().split(' ')
  tipo_prop=''
  
  for palabra in palabras:

    if palabra in tipo_propiedades:
        tipo_prop = palabra
        break
    else:
        tipo_prop = ''

  if tipo_prop == '':

    texto=np.array(lineas)[1]

    palabras=texto.lower().split(' ')
    tipo_prop=''
    
    for palabra in palabras:

      if palabra in tipo_propiedades:
          tipo_prop = palabra
          break
      else:
          #tipo de propiedad por defecto será apartamento
          tipo_prop = 'apartamento'

  return tipo_prop

In [ ]:
df_propiedades["municipio"]=df_propiedades[["titulo","descripcion"]].apply(extract_ubicacion,axis=1)

In [ ]:
df_propiedades["tipo_propiedad"]=df_propiedades[["titulo","descripcion"]].apply(extract_tipo, axis=1)

In [ ]:
df_propiedades.describe(include=['O'])

,tipo,url,titulo,precio,area_m2,habitaciones,banos,garajes,descripcion,caractint,caractext,caractsec,estrato,antiguedad,municipio,tipo_propiedad
count,2715,2715,2715,2715,2715,2715,2715,2715,2715,2715,2715,2715,2635,2188,2715,2715
unique,2,2715,1297,411,667,17,15,12,2624,1862,1678,578,7,6,11,5
top,Usada,https://www.fincaraiz.com.co/finca-en-venta/ri...,Apartamento en Venta Rionegro RIONEGRO,$ 320.000.000,"200,00 m²",3,2,1,6351568 Proyecto de apartamentos de 2 y 3 habi...,[],[],[],4,1 a 8 años,rionegro,casa
freq,2678,1,79,51,54,1153,1032,1200,10,302,426,450,1014,1167,1360,1189


In [ ]:
df_propiedades.info(show_counts=True, max_cols=190)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2715 entries, 0 to 2714
Data columns (total 190 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    tipo                         2715 non-null   object 
 1    url                          2715 non-null   object 
 2    titulo                       2715 non-null   object 
 3    precio                       2715 non-null   object 
 4    area_m2                      2715 non-null   object 
 5    habitaciones                 2715 non-null   object 
 6    banos                        2715 non-null   object 
 7    garajes                      2715 non-null   object 
 8    descripcion                  2715 non-null   object 
 9    balcon                       1401 non-null   float64
 10   bano_de_servicio             201 non-null    float64
 11   chimenea                     459 non-null    float64
 12   cocina_equipada              378 non-null    float64
 13   c

In [ ]:
df_propiedades['municipio'].value_counts()

In [ ]:
df_propiedades['tipo_propiedad'].value_counts()

In [ ]:
df_propiedades['tipo'].value_counts()

In [ ]:
list(df_propiedades[df_propiedades['tipo_propiedad']==""]["url"])

In [ ]:
#unificar apartamentos
df_propiedades['tipo_propiedad']=df_propiedades['tipo_propiedad'].apply(lambda x: x.replace("apartamentos", "apartamento"))


In [ ]:
df_propiedades['tipo_propiedad'].value_counts()

In [ ]:
def unifica_muni(texto):
  muni=''
  
  if texto=='retiro':
    muni="el retiro"

  elif texto=='ceja':
    muni="la ceja"
 
  elif texto=='viboral':
    muni="el carmen de viboral"
  
  elif texto=='llano':
    muni="llanogrande"

  elif texto=='unión':
    muni="la unión"

  elif texto=='pereira':
    muni="san antonio de pereira"

  else:
    return texto

  return muni

In [ ]:
df_propiedades['municipio']=df_propiedades['municipio'].apply(unifica_muni)

In [ ]:
df_propiedades['municipio'].value_counts()

In [ ]:
#Llenar valores nulos con cero
df_propiedades=df_propiedades.fillna(0)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [ ]:
#Transformar variable precio
df_propiedades['precio']=df_propiedades['precio'].apply(lambda x: x.replace('$','').replace('.','').replace(',','.').strip()).astype(float)

In [ ]:
#transformar variable área
df_propiedades['valida_ha']=df_propiedades['area_m2'].apply(lambda x: 1 if x.find("Ha")!=-1 else 0)

In [ ]:
#Limpiar texto y convertir a número
df_propiedades['area_m2'] = df_propiedades['area_m2'].apply(lambda x: x.replace('Ha','').replace('m2','').replace('m²','').replace('.','').replace(',','.').strip()).astype(float)

In [ ]:
#Corregir unidades hectareas
df_propiedades['area_m2'] = df_propiedades[['area_m2','valida_ha']].apply(lambda x: x.area_m2 if x.valida_ha == 0 else x.area_m2*10000,axis=1)

In [ ]:
df_propiedades.drop(['valida_ha'], axis=1, inplace=True)

In [ ]:
#Transformar variable habitaciones -- > El número por defecto será 1 sino especifica
df_propiedades['habitaciones']=df_propiedades['habitaciones'].apply(lambda x: x.replace('---','1').replace('Sin especificar','1').strip()).astype(float)

In [ ]:
#Transformar variable baños -- > El número por defecto será 1 sino especifica
df_propiedades['banos']=df_propiedades['banos'].apply(lambda x: x.replace('---','1').replace('Sin especificar','1').strip()).astype(float)

In [ ]:
#Transformar variable garajes -- > El número por defecto será 0 sino especifica
df_propiedades['garajes']=df_propiedades['garajes'].apply(lambda x: x.replace('---','0').replace('Sin especificar','0').replace('Más de 10','10').strip()).astype(float)

In [ ]:
df_propiedades['habitaciones'].value_counts()

In [ ]:
df_propiedades.info(show_counts=True, max_cols=190)

In [ ]:
df_propiedades=df_propiedades[df_propiedades["precio"]<=2500000000]

In [ ]:
df_propiedades.shape

(2478, 190)

In [ ]:
df_propiedades.to_csv('df_prop_clean.csv', header=True, index=False, sep=";")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.figure(figsize=(15,8))
ax = sns.boxplot(x="tipo_propiedad", y="precio", data=df_propiedades)

In [ ]:
plt.figure(figsize=(15,8))

ax = sns.boxplot(x="municipio", y="precio", data=df_propiedades)

In [ ]:
plt.figure(figsize=(15,8))
df_propiedades.precio.hist();

In [ ]:
list(df_propiedades[df_propiedades["precio"]==215000000000].url)

In [ ]:
df_propiedades["precio"].max()

In [ ]:
#Crear copia del dataframe y quitar columnas que no estarán en el modelo
data_model=df_propiedades.copy()
#columnas_quitar=['tipo','url','titulo','descripcion','caractint','caractext','caractsec']
columnas_quitar=['tipo','url','titulo','descripcion','caractint','caractext','caractsec']
data_model.drop(columnas_quitar, axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

In [ ]:
X = data_model.drop('precio', axis=1)
y=data_model['precio']

In [ ]:
X = pd.get_dummies(X, columns=['tipo_propiedad','municipio','antiguedad','estrato'], prefix=["tipo","muni","antig","estrato"] )

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

In [ ]:
model1 = DecisionTreeRegressor(random_state=0, max_depth=10, max_features=100)
model1.fit(X_train, y_train)

In [ ]:
print("The model training score is" , model1.score(X_train, y_train))
print("The model test score is" , model1.score(X_test, y_test))

In [ ]:
model2 = RandomForestRegressor(max_depth=10, random_state=0)
model2.fit(X_train, y_train)

In [ ]:
print("The model training score is" , model2.score(X_train, y_train))
print("The model test score is" , model2.score(X_test, y_test))

In [ ]:
parameters = {'n_estimators':[20, 50, 100,110,120,150], 'max_depth':[10,20,30,40]}
rf = RandomForestRegressor(max_depth=10, random_state=0)
clf = GridSearchCV(rf, parameters, n_jobs=4, verbose=8)
clf.fit(X_train, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
print("The model training score is" , clf.score(X_train, y_train))
print("The model test score is" , clf.score(X_test, y_test))

In [ ]:
pd.DataFrame(clf.predict(X_test))

In [ ]:
#y_pred = model1.predict(scaler.inverse_transform(X_test))

#mse = mean_squared_error(y_test, y_pred)
#print("RMSE: ", mse*(1/2.0)) 

In [ ]:
#poly_feature = PolynomialFeatures(degree=4,include_bias=False)
#x_poly = poly_feature.fit_transform(X_train)

In [ ]:
model2 = Ridge(solver='lsqr')
model2.fit(X_train, y_train)

In [ ]:
print("The model training score is" , model2.score(X_train, y_train))
print("The model test score is" , model2.score(X_test, y_test))

In [ ]:
y_pred = model2.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("RMSE: ", mse*(1/2.0)) 

In [ ]:
pd.DataFrame(y_pred).head(100)

Realizar análisis de errores para entender ls datos atipicos
Outliers
scaterplot real vs predicción
mean absolute error

In [ ]:
# Consultar al profe:

# Se puede tener metrica customizada para errores y que el modelo no genere valores de precio negativos
# Como se hacen las predicciones, hay que sacar el inverso de la tranformación, con standard scaler? 